<h1 align="center" style="color:green;font-size: 3em;">
Implementing Fine-tuning Techniques</h1>


Implementing various fine-tuning methods as described in different papers, specifically LoRA and IA3.

In this notebook, we will explore IA3 implementations:
- by modifying activations
- by manipulating weights and biases

### Install Dependencies

In [1]:
%pip install datasets -q

Note: you may need to restart the kernel to use updated packages.


### Import Libraries

In [1]:

# importing required libraries
import torch
import torch.nn as nn
import collections
import random
import numpy as np
import math
import matplotlib.pyplot as plt
import warnings

from torch.optim import AdamW
from typing import List
from torch.nn import functional as F
from tqdm import tqdm
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    DataCollatorForLanguageModeling,
    TrainingArguments,
)
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, T5Tokenizer, T5ForSequenceClassification
from torch.utils.data import DataLoader

warnings.simplefilter("ignore")
print(torch.__version__)

2.6.0+cu124


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### IA3 Memory Analysis

In this section, we will perform memory analysis on a well-known model from Hugging Face called Gemma. Gemma is a state-of-the-art model used for various natural language processing tasks.

However, it is a "gated model", so will require access.

To read more about how to get access to gated models: [link](https://huggingface.co/docs/hub/en/models-gated).

In [3]:
%pip install huggingface_hub -q

Note: you may need to restart the kernel to use updated packages.


In [7]:
from huggingface_hub import login
login()

In [8]:
## Load the gemma model (NEED AUTHENTICATIOIN)
model_name = "google/gemma-2b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]